In [27]:
import pandas as pd
import wandb
from datetime import datetime
import numpy as np


class Results():
    def __init__(self):
        num_seeds = 5
        columns = [f"nguyen_{i}" for i in range(1, 13, 1)]
        self.results_grammar_1_Classic = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_grammar_1_AmEx = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_grammar_2_Classic = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_grammar_2_AmEx = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_uniform_1_Classic = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_uniform_1_AmEx = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_uniform_2_Classic = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)
        self.results_uniform_2_AmEx = pd.DataFrame(index=list(range(1, num_seeds + 1, 1)), columns=columns)

    def fill_results(self, config, summary):

        seed = config['seed']
        data_path = config['data_path']
        engine = config['MCTS_engine']
        prior = config['prior_source']
        target_value = summary['time to perfect fit']
        dataset = config['data_path'].split('/')[1]

        if 'data_grammar_1' in data_path and engine == 'Endgame' and  prior =='grammar':
            self.results_grammar_1_AmEx.loc[seed, dataset] = target_value

        elif 'data_grammar_2' in data_path and engine == 'Endgame' and  prior =='grammar':
            self.results_grammar_2_AmEx.loc[seed, dataset] = target_value

        elif 'data_grammar_1' in data_path and engine == 'Classic' and  prior =='grammar':
            self.results_grammar_1_Classic.loc[seed, dataset] = target_value

        elif data_path == 'data_grammar_2' in data_path and engine == 'Classic' and  prior =='grammar':
            self.results_grammar_2_Classic.loc[seed, dataset] = target_value
            
        elif 'data_grammar_1' in data_path and engine == 'Endgame' and  prior =='uniform':
            self.results_uniform_1_AmEx.loc[seed, dataset] = target_value

        elif 'data_grammar_2' in data_path and engine == 'Endgame' and  prior =='uniform':
            self.results_uniform_2_AmEx.loc[seed, dataset] = target_value

        elif 'data_grammar_1' in data_path and engine == 'Classic' and  prior =='uniform':
            self.results_uniform_1_Classic.loc[seed, dataset] = target_value

        elif data_path == 'data_grammar_2' in data_path and engine == 'Classic' and  prior =='uniform':
            self.results_uniform_2_Classic.loc[seed, dataset] = target_value


api = wandb.Api()
entity, project = "jgu-wandb", "neural_guided_symbolic_regression_final"
# Example: January 1st, 2024

# created_at (str): ISO timestamp when the run was started
runs = api.runs(entity + "/" + project, filters={"tags": {"$in": ["no_net"]}})

summary_list, config_list, name_list = [], [], []
results_obj = Results()

for run in runs:
    # .summary contains output keys/values for
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files
    result_dict = run.summary._json_dict
    try:
        summary_dict = run.summary._json_dict
        config_dict = {k: v for k, v in run.config.items() if not k.startswith("_")}
        results_obj.fill_results(config_dict, summary_dict)
    except Exception as e:
        print(f"{e} their is a problem with {run.name} ")

print('end')


'time to perfect fit' their is a problem with no_net__uniform__data_grammar_2_nguyen_6__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_2_nguyen_6__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_2_nguyen_6__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_1_nguyen_4__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_1_nguyen_4__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_1_nguyen_4__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_1_nguyen_4__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no_net__uniform__data_grammar_1_nguyen_4__MeasurementEncoderDummy__None 
'time to perfect fit' their is a problem with no

In [18]:
df= pd.DataFrame(results_obj.results_grammar_1_AmEx)
df

,nguyen_1,nguyen_2,nguyen_3,nguyen_4,nguyen_5,nguyen_6,nguyen_7,nguyen_8,nguyen_9,nguyen_10,nguyen_11,nguyen_12
1,24717,358210,NaN,NaN,7438,NaN,102753,69813,301684,NaN,515,NaN
2,7970,NaN,NaN,NaN,159927,NaN,31915,6805,NaN,NaN,680,NaN
3,21537,NaN,NaN,NaN,22768,NaN,9576,89140,NaN,NaN,549,NaN
4,21891,NaN,NaN,NaN,95967,NaN,53164,94275,110349,NaN,640,NaN
5,20798,NaN,NaN,NaN,43056,NaN,28539,34221,NaN,NaN,622,NaN


In [29]:
import pandas as pd
df= pd.DataFrame(results_obj.results_uniform_2_AmEx)
#df_grammar
nan_values = df.isna().sum().sum()
#nan_values = df.isna().sum(axis=0)
#mean_values = df.mean(axis=0).round(0)
mean_values = df.mean(axis=0).mean(axis=0).round(0)
#mean_values =mean_values.astype(int)
# for nguyen in list(mean_values.index): 
print(f"results_uniform_2_AmEx:    {mean_values} , {nan_values}")
#df_grammar.mean(axis=0)

results_uniform_2_AmEx:    645.0 , 15
